In [599]:
import numpy as np
import pandas as pd
import os

In [626]:
## Making a fit function by first taking all the files from the training data
## Making stop words 

entries = os.listdir('Desktop/naivebayes/20_newsgroups')
target_dict = {}
for i in entries:
    target_dict[i] = entries.index(i)
f = open('Desktop/naivebayes/stopwords.txt')
list_wrds = f.read()
stop_words = list_wrds.split('\n')



In [601]:
## Making a feature dictionary here 
## choosing the first 5000 words for training purpose 

word_dict= {}
w_dict = {}
string_to_omit ="! @ # $ % ^ & * ( ) 1 2 3 4 5 6 7 8 9 0 ' , { . < > /  = \ + _ ? - | : ; + ="
string_to_omit_lis = string_to_omit.split()
string_to_omit_lis = string_to_omit_lis + list("'")+list('"')
for i in entries:
    files_of_a_topic = os.listdir('Desktop/naivebayes/20_newsgroups/'+i)
    word_dict = {}
    for z in files_of_a_topic:
        file = open('Desktop/naivebayes/20_newsgroups/'+i+'/'+z)
        list_of_words_in_file = file.read().split()
        for word_in_file in list_of_words_in_file:
            if word_in_file in stop_words:
                continue
            elif word_in_file in string_to_omit:
                continue
            else:
                if word_in_file in word_dict:
                    word_dict[word_in_file] = word_dict[word_in_file]+1
                else:
                    p = 0 
                    lis_to_check = list(word_in_file)
                    for o in lis_to_check:
                        if o in string_to_omit_lis :
                            p = p-1
                    if p==0:
                        word_dict[word_in_file] =1
    w_dict.update(word_dict)

In [682]:
## chosing first 5000 words with most frequent occurences

sorted(word_dict.items(), key=lambda x: x[1], reverse=True)
lis = list(w_dict)

features = list(lis[:5000])
len(features)


5000

In [683]:
## Making the training data

X = [[0]*5001]*19997
X = np.array(X)
row = 0 
for class_val in entries:
    files_of_a_topic = os.listdir('Desktop/naivebayes/20_newsgroups/'+class_val)
    for doc in files_of_a_topic:
        file = open('Desktop/naivebayes/20_newsgroups/'+class_val+'/'+doc)
        list_of_words_in_file = file.read().split()
        for word_in_file in list_of_words_in_file:
            if word_in_file in features:
                X[row][features.index(word_in_file)]  = X[row][features.index(word_in_file)] + 1
        X[row][5000] = target_dict[class_val]
        row = row + 1

In [684]:
## Making the probability dictionary with each class as key 
## each class has another dictionary having the frequency of occurence of a particular feature for a given word in vocabulary 
   #for that particular doc of that class
    
probability_dict = {}
for class_val in range(20):
    probability_dict[class_val] = {}
feature_dict = {}
val = 0
for i in features:
    feature_dict[i] = val
    val = val+1
X_train = pd.DataFrame(X,columns=features+['cl'])

## Making X train and Y Train 

classes = set(Y_train)
for class_val in classes:
    sum_of_all_points_in_a_class = 0
    X_t = X_train[X_train.iloc[:,5000] == class_val]
    b = probability_dict[class_val]
    for feature in features:
        freq = X_t.loc[:,feature].sum()
        b[feature_dict[feature]] = freq
        sum_of_all_points_in_a_class = sum_of_all_points_in_a_class + freq
    b['total_points'] = sum_of_all_points_in_a_class

In [685]:
probability_dict

Y_t = X_train.loc[:,'cl']

X_t = X_train.drop('cl',axis= 1)


In [686]:
## Predicting output for out testing data

def probability(list_of_words_in_file , class_val , proability_dict,features,Y_train):
    denominator = probability_dict[class_val]['total_points']
    x = 0 
    for i in Y_t:
        if i == class_val:
            x+=1
            
    output = np.log(x) - np.log(len(Y_train))
    ans = -11111111
    for word in list_of_words_in_file:
        if word not in features:
            continue
        else:
            ##Applying Bayesian probability formula for logarithmic calculations and also implementing the laplacian correction
            
            current_word_probability = np.log(probability_dict[class_val][features.index(word)]+ 1) - np.log(denominator + 5000)
            output = output + current_word_probability
    ans = class_val
    return output , ans
            
## predict function for a single point in test data 

def predict(file , probability_dict,features,Y_train):
    list_of_words_in_file = file.read().split()
    answer = -1000000
    best_probability = -100000
    run = True
    for class_val in range(20):
        probability_ans , ans = probability(list_of_words_in_file ,class_val,probability_dict,features,Y_train)
        if(run or probability_ans>best_probability):
            best_probability = probability_ans
            answer = ans
        run = False
    return answer
    

    
## going through test data to get predictions using predict function

entries_test = os.listdir('Desktop/naivebayes/mini_newsgroups')
string_to_omit_lis = string_to_omit_lis + list("'")+list('"')
y_pred = []

for i in entries_test:
    files_of_a_topic_to_test = os.listdir('Desktop/naivebayes/mini_newsgroups/'+i)
    for z in files_of_a_topic_to_test:
        file = open('Desktop/naivebayes/mini_newsgroups/'+i+'/'+z)
        y_pred.append(predict(file , probability_dict,features , Y_t))
        

y_pred
    
    

[18,
 0,
 0,
 16,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 19,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 6,
 0,
 0,
 19,
 6,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 15,
 19,
 0,
 0,
 0,
 0,
 8,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 19,
 19,
 19,
 18,
 0,
 0,
 12,
 0,
 0,
 0,
 19,
 0,
 0,
 19,
 0,
 18,
 10,
 0,
 0,
 0,
 0,
 19,
 5,
 1,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 3,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 9,
 1,
 1,
 12,
 1,
 1,
 1,
 1,
 12,
 9,
 2,
 2,
 8,
 1,
 1,
 2,
 1,
 1,
 5,
 1,
 1,
 2,
 2,
 5,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 8,
 2,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 12,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 2,
 1,
 1,
 6,
 1,
 7,
 11,
 1,
 4,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 2,
 2,
 14,
 14,
 6,
 2,
 2,
 2,
 1,
 3,
 2,
 2,
 9,
 2,
 2,
 2,
 5,
 2,
 3,
 2,
 2,
 2,
 9,
 11,
 1

In [697]:
## So accuracy of the self implemented model can now be calculated
acc_lis = []
for i in range(20):
    acc_lis = acc_lis +  [i] *100
    
## acc_lis is the actual Y_values for test data
    
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_pred,acc_lis)
accuracy


0.7675

In [ ]:
## Accuracy of self implemented model is 0.765

In [699]:
##implementing sklearn model for same data 

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_t, Y_t)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [700]:
##Making test data set

X_test = [[0]*5000]*2000
X_test = np.array(X_test)
row = 0 
for class_val in entries_test:
    files_of_a_topic = os.listdir('Desktop/naivebayes/mini_newsgroups/'+class_val)
    for doc in files_of_a_topic:
        file = open('Desktop/naivebayes/mini_newsgroups/'+class_val+'/'+doc)
        list_of_words_in_file = file.read().split()
        for word_in_file in list_of_words_in_file:
            if word_in_file in features:
                X_test[row][features.index(word_in_file)]  = X_test[row][features.index(word_in_file)] + 1
        row = row + 1
X_test = pd.DataFrame(X_test,columns=features)

In [703]:
## using predict function 

Y_test_pred  = clf.predict(X_test)

ac = accuracy_score(acc_lis,Y_test_pred)
print(ac)


0.7675


array([18,  0,  0, ..., 19, 19, 19])

In [ ]:
##Accuracy of sklearn model is 0.765